In [1]:
!wget https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/shahname.csv

--2024-06-13 11:47:11--  https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/shahname.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5609688 (5.3M) [text/plain]
Saving to: ‘shahname.csv’

shahname.csv        100%[===================>]   5.35M  --.-KB/s    in 0.04s   

2024-06-13 11:47:12 (146 MB/s) - ‘shahname.csv’ saved [5609688/5609688]



In [2]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from tensorflow import keras

**Load the dataset**

In [3]:
df = pd.read_csv('/content/shahname.csv')
text = '\n'.join(df['Text'])

**Display the first 250 characters of the text**

In [4]:
print(text[:250])

به نام خداوند جان و خرد
کز این برتر اندیشه بر نگذرد
خداوند نام و خداوند جای
خداوند روزی ده رهنمای
خداوند کیوان و گَردان سپهر
فروزندهٔ ماه و ناهید و مهر
ز نام و نشان و گمان برتر است
نگارندهٔ بر شده پیکر است
به بینندگان آفریننده را
نبینی مرنجان دو بینن


**Create a vocabulary of unique characters**

In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
for v in vocab:
    print(f'{v}', end=' ')

57 unique characters

   ! ( ) :   « » ، ؛ ؟ ء آ أ ؤ ئ ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن ه و ي َ ُ ِ ّ ْ ٔ پ چ ژ ک گ ی ‌ 

**Mapping from characters to IDs and vice versa**

In [6]:
ids_to_chars = keras.layers.StringLookup(vocabulary=vocab, invert=True, mask_token=None)
ids_from_chars = keras.layers.StringLookup(vocabulary=vocab, invert=False, mask_token=None)

**Display the vocabulary**

In [7]:
for v in ids_from_chars.get_vocabulary():
    print(v, end=' ')

[UNK] 
   ! ( ) :   « » ، ؛ ؟ ء آ أ ؤ ئ ا ب ت ث ج ح خ د ذ ر ز س ش ص ض ط ظ ع غ ف ق ل م ن ه و ي َ ُ ِ ّ ْ ٔ پ چ ژ ک گ ی ‌ 

**Convert text to sequence of IDs**

In [8]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

**Define sequence length and batch size**

In [9]:
SEQUENCE_LENGTH = 100
BATCH_SIZE = 64
AUTOTUNE = tf.data.experimental.AUTOTUNE

**Create a dataset of sequences**

In [10]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
sequences = ids_dataset.batch(SEQUENCE_LENGTH + 1, drop_remainder=True, num_parallel_calls=AUTOTUNE)

**Split input and target texts**

In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target, num_parallel_calls=AUTOTUNE)

**Create training batches**

In [12]:
dataset = dataset.batch(BATCH_SIZE, num_parallel_calls=AUTOTUNE, drop_remainder=True)
dataset = dataset.prefetch(AUTOTUNE)

**Model parameters**

In [13]:
VOCAB_SIZE = len(ids_from_chars.get_vocabulary())
EMBEDDING_DIM = 10
RNN_UNITS = 2048

**Define the model**

In [14]:
class MyModel(keras.Model):
    def __init__(self, vocabulary_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)
        self.gru = keras.layers.GRU(units=rnn_units, return_sequences=True, return_state=True)
        self.dense = keras.layers.Dense(vocabulary_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        if return_state:
            return x, states
        return x

In [15]:
model = MyModel(vocabulary_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, rnn_units=RNN_UNITS)

sample_input = tf.random.uniform((BATCH_SIZE, SEQUENCE_LENGTH), dtype=tf.int32, minval=0, maxval=VOCAB_SIZE)
model(sample_input)

<tf.Tensor: shape=(64, 100, 58), dtype=float32, numpy=
array([[[-5.7661737e-04, -5.2061817e-04, -8.8580680e-04, ...,
          5.2249507e-04,  1.1439670e-03, -7.4360811e-04],
        [-3.1005214e-03, -1.9589346e-03, -3.2850323e-04, ...,
         -3.6103535e-04, -4.3967448e-04, -1.9227804e-03],
        [-2.5325555e-03,  3.5561589e-04, -1.7043864e-03, ...,
          6.0410355e-05, -2.1175449e-03,  1.2615696e-04],
        ...,
        [-1.2644095e-03,  1.7504464e-03, -1.1788621e-03, ...,
         -1.0697353e-03,  4.8672577e-05, -1.7204704e-03],
        [-1.3111050e-03,  5.0596666e-04, -3.7897439e-04, ...,
          1.2282966e-04,  1.4894943e-03,  7.2462717e-04],
        [ 7.6025550e-04, -2.1489503e-04, -1.2188213e-04, ...,
         -1.9392715e-04,  1.2229050e-03,  2.0593975e-03]],

       [[-3.3872670e-03, -1.7113626e-03,  1.6196291e-03, ...,
         -4.9119152e-04,  2.0570228e-04, -2.4463406e-03],
        [-1.0136575e-03, -1.6698710e-04, -1.1979711e-05, ...,
         -5.9060310e-04,  7.

In [16]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  580       
                                                                 
 gru (GRU)                   multiple                  12656640  
                                                                 
 dense (Dense)               multiple                  118842    
                                                                 
Total params: 12776062 (48.74 MB)
Trainable params: 12776062 (48.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

**Set up checkpoints**

In [18]:
checkpoints_dir = './temp/chpts/'
checkpoint_prefix = os.path.join(checkpoints_dir, 'chpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

**Train the model**

In [19]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
396/396 [==============================] - 74s 179ms/step - loss: 2.6356
Epoch 2/30
396/396 [==============================] - 72s 182ms/step - loss: 1.9355
Epoch 3/30
396/396 [==============================] - 72s 183ms/step - loss: 1.6278
Epoch 4/30
396/396 [==============================] - 72s 183ms/step - loss: 1.4654
Epoch 5/30
396/396 [==============================] - 72s 183ms/step - loss: 1.3530
Epoch 6/30
396/396 [==============================] - 73s 184ms/step - loss: 1.2590
Epoch 7/30
396/396 [==============================] - 72s 183ms/step - loss: 1.1661
Epoch 8/30
396/396 [==============================] - 72s 183ms/step - loss: 1.0665
Epoch 9/30
396/396 [==============================] - 72s 183ms/step - loss: 0.9716
Epoch 10/30
396/396 [==============================] - 73s 184ms/step - loss: 0.9135
Epoch 11/30
396/396 [==============================] - 72s 183ms/step - loss: 0.8789
Epoch 12/30
396/396 [==============================] - 72s 183ms/step - lo

**Define a class for generating text**

In [20]:
class OneStep(tf.keras.Model):
    def __init__(self, model, ids_to_chars, ids_from_chars, temperature=1.0):
        super(OneStep, self).__init__()
        self.temperature = temperature
        self.model = model
        self.ids_to_chars = ids_to_chars
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :] / self.temperature
        predicted_logits += self.prediction_mask
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.ids_to_chars(predicted_ids)
        return predicted_chars, states

**Generate and display text**

In [21]:
one_step_model = OneStep(model, ids_to_chars, ids_from_chars, temperature=0.75)
start = time.time()
states = None
next_char = tf.constant(['به نام خداوند'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

# Display generated text and runtime
print(result[0].numpy().decode('utf-8'), '\n\n\n')
print('\nRun time:', end - start)

به نام خداوند جای
یکی نامه‌ای بن ببین جایگاه
سپهبد چو بشنید دینار او
ز گلبرگشاهن گذشت ارجمند
به پیش سراپرده‌ام بنده‌ایم
که فرزند من زاد فرزند خویش
کجا هم به بی‌تان و تیغی ودی
بتار کمان و به سوی نشی
تو من در هله خیره دینار دفت
برین گونه یاری  زو یاد کرد
ز جایی که گفتند ما بی‌هرربه سرد
شود جوشن و ترگ پر آرده دروی
که این مهره را تا ده و دو هزار
سساپند و گردنکشی داده‌ایم
دل از باستان پیش خسرو دوید
کمان را بزه کرد و دل کرده دید
همه کاخ و از تو پر آب گشت
پییده شدی پیش و پیل دهم
یکی مهتری بر در شهریار
فرستاد و در شهر نه مردِ بود
سخن هرچه پرسیم کاووس کن
بزرگان و فرزند روشن توبر
نه آن رفتگان را ز تخم پدر
که چون او منم سایهٔ فر تو
بباشیم تا نام تو سر به چرخ
به بالین نهاد آن گرامی  زود
که خیزد پر از آب رومه به مخت
همه یک بیک اندر آورد شاد
در گنج بد شاد و بالا گرفت
سپهبد چو برخواند آن توبگ تست
سوی خیره زان دید برگشت شاه
دلیری صمین سی هنرمار شاه
ز قلب سپاه اندر آمد به جای
چو پیروز بازخ سپیده دمان
همی تیر بارید گودرز خاک
همی‌داد چون کاخ او خور شاه
اگر بد سلیحان نه آیین بدند
دگر ناز روز به آن شاختر
ب